In [2]:
import os

print(os.getcwd())

/home/abner/work/kaldi/egs/torgo-speech_processing/asr


In [3]:
!cd ./data && tree

.
├── test
│   ├── spk2utt
│   ├── text
│   ├── utt2spk
│   └── wav.scp
└── train
    ├── spk2utt
    ├── text
    ├── utt2spk
    └── wav.scp

2 directories, 8 files


## Stage 1 -- Language Model Building

### Prepare ARPA LM and vocabulary using SRILM
```
local/torgo_prepare_lm.sh --order ${lm_order} || exit 1
```
#### Pronunciations for OOV words are obtained using a pre-trained Sequitur model
```
local/torgo_prepare_dict.sh 

utils/prepare_lang.sh --position-dependent-phones $pos_dep_phones \
        data/local/dict '!SIL' data/local/lang data/lang || exit 1
```
### Prepare G.fst and data {train,test} directories
```
local/torgo_prepare_grammar.sh "test" || exit 1
```

In [4]:
!./run.sh


===== PREPARING Language DATA =====


=== Building a language model ...

=== Preparing the dictionary ...

--- Downloading CMU dictionary ...
A    data/local/dict/cmudict/scripts
A    data/local/dict/cmudict/sphinxdict
A    data/local/dict/cmudict/00README_FIRST.txt
A    data/local/dict/cmudict/cmudict-0.7b
A    data/local/dict/cmudict/scripts/test_cmudict.pl
A    data/local/dict/cmudict/scripts/README.txt
A    data/local/dict/cmudict/scripts/make_baseform.pl
A    data/local/dict/cmudict/scripts/CompileDictionary.sh
A    data/local/dict/cmudict/scripts/sort_cmudict.pl
A    data/local/dict/cmudict/scripts/test_dict.pl
A    data/local/dict/cmudict/cmudict.0.6d
A    data/local/dict/cmudict/cmudict.0.7a
A    data/local/dict/cmudict/cmudict-0.7b.phones
A    data/local/dict/cmudict/cmudict-0.7b.symbols
A    data/local/dict/cmudict/sphinxdict/cmudict.0.7a_SPHINX_40
A    data/local/dict/cmudict/sphinxdict/cmudict_SPHINX_40
A    data/local/dict/cmudict/sphinxdict/README.txt
A    data/local/dic

## Stage 2 -- Feature Extraction

```
mfccdir=mfcc
utils/validate_data_dir.sh data/train
utils/fix_data_dir.sh data/train
utils/validate_data_dir.sh data/test
utils/fix_data_dir.sh data/test


steps/make_mfcc.sh --nj $nj --cmd "$train_cmd" data/train exp/make_mfcc/train $mfccdir
steps/make_mfcc.sh --nj 1 --cmd "$train_cmd" data/test exp/make_mfcc/test $mfccdir
```
# Making cmvn.scp files
```
steps/compute_cmvn_stats.sh data/train exp/make_mfcc/train $mfccdir
steps/compute_cmvn_stats.sh data/test exp/make_mfcc/test $mfccdir
```

In [5]:
!./run.sh


===== FEATURES EXTRACTION =====

utils/validate_data_dir.sh: no such file data/train/feats.scp (if this is by design, specify --no-feats)
fix_data_dir.sh: kept all 15475 utterances.
fix_data_dir.sh: old files are kept in data/train/.backup
utils/validate_data_dir.sh: WARNING: you have only one speaker.  This probably a bad idea.
   Search for the word 'bold' in http://kaldi-asr.org/doc/data_prep.html
   for more information.
utils/validate_data_dir.sh: no such file data/test/feats.scp (if this is by design, specify --no-feats)
fix_data_dir.sh: kept all 1087 utterances.
fix_data_dir.sh: old files are kept in data/test/.backup
steps/make_mfcc.sh --nj 6 --cmd run.pl data/train exp/make_mfcc/train mfcc
utils/validate_data_dir.sh: Successfully validated data-directory data/train
steps/make_mfcc.sh: [info]: no segments file exists: assuming wav.scp indexed by utterance.
steps/make_mfcc.sh: Succeeded creating MFCC features for train
steps/make_mfcc.sh --nj 1 --cmd run.pl data/test exp/make_m

## Stage 3 -- GMM-HMM Acoustic Model Training 

1. Monophone training + alignment
```
steps/train_mono.sh --nj $nj --cmd "$train_cmd" --cmvn-opts "$cmvn_opts" \
        $data_dir/train $lang $exp_dir/mono
steps/align_si.sh --nj $nj --cmd "$train_cmd" 
```

2. Delta training
```
steps/train_deltas.sh --cmd "$train_cmd" --cmvn-opts "$cmvn_opts" \
        $Leaves $Gauss $data_dir/train $lang $exp_dir/mono_ali $exp_dir/tri1
steps/align_si.sh --nj $nj --cmd "$train_cmd" \
        $data_dir/train $lang $exp_dir/tri1 $exp_dir/tri1_ali
```

3. Delta-Delta training
```
steps/train_deltas.sh --cmd "$train_cmd" --cmvn-opts "$cmvn_opts" \
        $Leaves $Gauss $data_dir/train $lang $exp_dir/tri1_ali $exp_dir/tri2
steps/align_si.sh --nj $nj --cmd "$train_cmd" \
        $data_dir/train $lang $exp_dir/tri2 $exp_dir/tri2_ali
```

4. LDA-MLLT training
```
steps/train_lda_mllt.sh --cmd "$train_cmd" --cmvn-opts "$cmvn_opts" \
        $Leaves $Gauss $data_dir/train $lang $exp_dir/tri2_ali $exp_dir/tri3
steps/align_si.sh --nj $nj --cmd "$train_cmd" \
        $data_dir/train $lang $exp_dir/tri3 $exp_dir/tri3_ali
```

5. SAT training
```
steps/train_sat.sh --cmd "$train_cmd" \
        $Leaves $Gauss $data_dir/train $lang $exp_dir/tri3_ali $exp_dir/tri4 
steps/align_fmllr.sh --nj $nj --cmd "$train_cmd" \
        $data_dir/train $lang $exp_dir/tri4 $exp_dir/tri4_ali
```

In [7]:
!./run.sh

steps/train_mono.sh --nj 6 --cmd run.pl --cmvn-opts  /home/abner/work/kaldi/egs/torgo-speech_processing/asr/data/train /home/abner/work/kaldi/egs/torgo-speech_processing/asr/data/lang /home/abner/work/kaldi/egs/torgo-speech_processing/asr/exp/mono
steps/train_mono.sh: Initializing monophone system.
steps/train_mono.sh: Compiling training graphs
steps/train_mono.sh: Aligning data equally (pass 0)
steps/train_mono.sh: Pass 1
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 2
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 3
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 4
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 5
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 6
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 7
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 8
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 9
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 10
steps

## Stage 4 -- Decoding 

```
utils/mkgraph.sh $lang_test $exp_dir/tri4 $exp_dir/tri4/graph
steps/decode_fmllr.sh --config conf/decode.config --nj 1 --cmd "$decode_cmd"  --num-threads $thread_nj --scoring_opts "$scoring_opts" \
        $exp_dir/tri4/graph $data_dir/test $exp_dir/tri4/decode
```

In [8]:
!./run.sh

tree-info /home/abner/work/kaldi/egs/torgo-speech_processing/asr/exp/tri4/tree 
tree-info /home/abner/work/kaldi/egs/torgo-speech_processing/asr/exp/tri4/tree 
fstpushspecial 
fstminimizeencoded 
fstdeterminizestar --use-log=true 
fsttablecompose /home/abner/work/kaldi/egs/torgo-speech_processing/asr/data/lang_test/L_disambig.fst /home/abner/work/kaldi/egs/torgo-speech_processing/asr/data/lang_test/G.fst 
fstisstochastic /home/abner/work/kaldi/egs/torgo-speech_processing/asr/data/lang_test/tmp/LG.fst 
-0.0672263 -0.0677127
[info]: LG not stochastic.
fstcomposecontext --context-size=3 --central-position=1 --read-disambig-syms=/home/abner/work/kaldi/egs/torgo-speech_processing/asr/data/lang_test/phones/disambig.int --write-disambig-syms=/home/abner/work/kaldi/egs/torgo-speech_processing/asr/data/lang_test/tmp/disambig_ilabels_3_1.int /home/abner/work/kaldi/egs/torgo-speech_processing/asr/data/lang_test/tmp/ilabels_3_1.75992 /home/abner/work/kaldi/egs/torgo-speech_processing/asr/data/lang

In [9]:
## WER's can be check in the exp/tri4/decode folder 
!ls exp/tri4/decode

lat.1.gz       wer_10_1.0  wer_13_0.0  wer_15_0.5  wer_17_1.0  wer_9_0.0
log	       wer_11_0.0  wer_13_0.5  wer_15_1.0  wer_7_0.0   wer_9_0.5
num_jobs       wer_11_0.5  wer_13_1.0  wer_16_0.0  wer_7_0.5   wer_9_1.0
scoring_kaldi  wer_11_1.0  wer_14_0.0  wer_16_0.5  wer_7_1.0
trans.1        wer_12_0.0  wer_14_0.5  wer_16_1.0  wer_8_0.0
wer_10_0.0     wer_12_0.5  wer_14_1.0  wer_17_0.0  wer_8_0.5
wer_10_0.5     wer_12_1.0  wer_15_0.0  wer_17_0.5  wer_8_1.0


In [10]:
# The best WER can be seen below
!cat exp/tri4/decode/scoring_kaldi/best_wer

%WER 36.10 [ 1005 / 2784, 247 ins, 101 del, 657 sub ] /home/abner/work/kaldi/egs/torgo-speech_processing/asr/exp/tri4/decode/wer_17_0.0


### The WER is not so great..
### Let's try to improve (lower) the WER

- Apply speed pertubation to training set.
    - see https://github.com/kaldi-asr/kaldi/blob/master/egs/wsj/s5/utils/data/perturb_data_dir_speed_3way.sh
    - It may be worth knowing what kind of speed pertubation would be most helpful when trying to improve cerebral palsy based dysarthric speech. Hint* it tends to be slower.
- Train higher order n-grams.
- Optional: run stage 5, which is a simple DNN implementation in Kaldi.
    - Next exercise we'll use a Pytorch-based DNN instead.